# Original model

In [ ]:
import argparse
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import numpy as np
import copy
import datasets
import models
from lib.utils import AverageMeter
from lib.protocols import *
import math
import warnings


import torch.nn.functional as F
from randaugment import RandAugmentMC
warnings.filterwarnings("ignore")

In [ ]:
def config():
    global args
    parser = argparse.ArgumentParser(description='config for RUC')
    parser.add_argument('--lr', default=0.01, type=float, metavar='LR', help='initial learning rate')
    parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum')
    parser.add_argument('--weight_decay', '--wd', default=5e-4, type=float, metavar='W', help='weight decay')
    parser.add_argument('--epochs', default=200, type=int, help='max epoch per round. (default: 200)')
    parser.add_argument('--batch_size', default=256, type=int, metavar='B', help='training batch size')
    parser.add_argument('--s_thr', default=0.99, type=float, help='confidence sampling threshold')
    parser.add_argument('--n_num', default=100, type=float, help='the number of neighbor')
    parser.add_argument('--o_model', default='checkpoint/selflabel_cifar-10.pth.tar', type=str, help='original model path')
    parser.add_argument('--e_model', default='checkpoint/simclr_cifar-10.pth.tar', type=str, help='embedding model save path')
    parser.add_argument('--seed', default=1567010775, type=int, help='random seed')
    
    args = parser.parse_args(args=[])
    return args

In [ ]:
class LabelSmoothLoss(nn.Module):
    
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1).long(), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss
    
LSloss = LabelSmoothLoss(0.5)

In [ ]:
def linear_rampup(current, rampup_length=200):
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip((current) / rampup_length, 0.1, 1.0)
        return float(current)

In [ ]:
class criterion_rb(object):
    def __call__(self, outputs_x, targets_x, outputs_u, targets_u, epoch):
        # Clean sample Loss
        probs_u = torch.softmax(outputs_u, dim=1)
        Lx = -torch.mean(torch.sum(F.log_softmax(outputs_x, dim=1) * targets_x, dim=1))
        Lu = 25*torch.mean((probs_u - targets_u)**2)
        Lu = linear_rampup(epoch) * Lu
        return Lx, Lu

def get_threshold(current):
    return 0.9 + 0.02*int(current / 40)
        
def extract_metric(net, p_label, evalloader, n_num):
    net.eval()
    feature_bank = []
    with torch.no_grad():
        for batch_idx, (inputs1 , _, _, _, indexes) in enumerate(evalloader):
            out = net(inputs1.cuda())
            feature_bank.append(out)
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        sim_indices_list = []
        for batch_idx, (inputs1 , _, _, _, indexes) in enumerate(evalloader):
            out = net(inputs1.cuda(non_blocking=True))
            sim_matrix = torch.mm(out, feature_bank)
            _, sim_indices = sim_matrix.topk(k=n_num, dim=-1)
            sim_indices_list.append(sim_indices)
        feature_labels = p_label.cuda()
        first = True
        count = 0
        clean_num = 0
        correct_num = 0
        for batch_idx, (inputs1 , _, _, targets, indexes) in enumerate(evalloader):
            labels = p_label[indexes].cuda().long()
            sim_indices = sim_indices_list[count]
            sim_labels = torch.gather(feature_labels.expand(inputs1.size(0), -1), dim=-1, index=sim_indices)
            # counts for each class
            one_hot_label = torch.zeros(inputs1.size(0) * sim_indices.size(1), 10).cuda()
            one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1).long(), value=1.0)
            pred_scores = torch.sum(one_hot_label.view(inputs1.size(0), -1, 10), dim=1)
            count += 1
            pred_labels = pred_scores.argsort(dim=-1, descending=True)
            prob, _ = torch.max(F.softmax(pred_scores, dim=-1), 1)   
            # Check whether prediction and current label are same
            noisy_label = labels
            s_idx1 = (pred_labels[:, :1].float() == labels.unsqueeze(dim=-1).float()).any(dim=-1).float()
            s_idx = (s_idx1 == 1.0)
            clean_num += labels[s_idx].shape[0]
            correct_num += torch.sum((labels[s_idx].float() == targets[s_idx].cuda().float())).item()

            if first:
                prob_set = prob
                pred_same_label_set = s_idx
                first = False
            else:
                prob_set = torch.cat((prob_set, prob), dim = 0)
                pred_same_label_set = torch.cat((pred_same_label_set, s_idx), dim = 0)

        print(correct_num, clean_num)
        return pred_same_label_set
            
def extract_confidence(net, p_label, evalloader, threshold):
    net.eval()
    devide = torch.tensor([]).cuda()
    clean_num = 0
    correct_num = 0
    for batch_idx, (inputs1, _, _, targets, indexes) in enumerate(evalloader):
        inputs1, targets = inputs1.cuda(), targets.cuda().float()
        labels = p_label[indexes].float()
        logits = net(inputs1)
        prob = torch.softmax(logits.detach_(), dim=-1)
        max_probs, _ = torch.max(prob, dim=-1)
        mask = max_probs.ge(threshold).float()
        devide = torch.cat([devide, mask])
        s_idx = (mask == 1)
        clean_num += labels[s_idx].shape[0]
        correct_num += torch.sum((labels[s_idx] == targets[s_idx])).item()
    
    print(correct_num, clean_num)
    return devide

def extract_hybrid(devide1, devide2, p_label, evalloader):
    devide = (devide1.float() + devide2.float() == 2)
    clean_num = 0
    correct_num = 0
    for batch_idx, (inputs1, _, _, targets, indexes) in enumerate(evalloader):
        inputs1, targets = inputs1.cuda(), targets.cuda().float()
        labels = p_label[indexes].float()
        mask = devide[indexes]
        s_idx = (mask == 1)
        clean_num += labels[s_idx].shape[0]
        correct_num += torch.sum((labels[s_idx] == targets[s_idx])).item()
    
    print(correct_num, clean_num)
    return devide

In [ ]:
def preprocess(args):
    mean = (0.4914, 0.4822, 0.4465)
    std = (0.2023, 0.1994, 0.2010)
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(size=32, scale=(0.2,1.)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])

    transform_test = transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])
    
    transform_strong = transforms.Compose([
            transforms.RandomResizedCrop(size=32, scale=(0.2,1.)),
            transforms.RandomHorizontalFlip(),
            RandAugmentMC(n=2, m=2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)])

    trainset = datasets.CIFAR10RUC(root="./data", transform=transform_test, transform2 = transform_train, transform3 = transform_train, transform4 = transform_strong, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True, num_workers=4, drop_last=False)
    testset = datasets.CIFAR10RUC(root="./data",transform=transform_test, transform2 = transform_test, transform3 = transform_test,  download=True)
    evalloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)
        
    return trainset, trainloader, testset, evalloader, 10

In [ ]:
def adjust_learning_rate(args, optimizer, epoch):
    # cosine learning rate schedule
    lr = args.lr
    lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:

def train(epoch, net, net2, trainloader, optimizer, criterion_rb, devide, p_label, conf):
    train_loss = AverageMeter()
    net.train()
    net2.train()
    
    num_iter = (len(trainloader.dataset)//args.batch_size)+1
    # adjust learning rate
    adjust_learning_rate(args, optimizer, epoch)  
    optimizer.zero_grad()
    correct_u = 0
    unsupervised = 0
    conf_self = torch.zeros(50000)
    for batch_idx, (inputs1 , inputs2, inputs3, inputs4, targets, indexes) in enumerate(trainloader):
        inputs1, inputs2, inputs3, inputs4, targets = inputs1.float().cuda(), inputs2.float().cuda(), inputs3.float().cuda(), inputs4.float().cuda(), targets.cuda().long()
        s_idx = (devide[indexes] == 1)
        u_idx = (devide[indexes] == 0)
        labels = p_label[indexes].cuda().long()
        labels_x = torch.tensor(p_label[indexes][s_idx]).squeeze().long().cpu()
        target_x = torch.zeros(labels_x.shape[0], 10).scatter_(1, labels_x.view(-1,1), 1).float().cuda()
        
        logit_o, logit_w1, logit_w2, logit_s = net(inputs1), net(inputs2), net(inputs3), net(inputs4)
        logit_s = logit_s[s_idx]
        max_probs, _ = torch.max(torch.softmax(logit_o, dim=1), dim=-1)
        conf_self[indexes] = max_probs.detach().cpu()
        optimizer.zero_grad()
        
        with torch.no_grad():
            # compute guessed labels of unlabel samples
            outputs_u11 = logit_w1[u_idx]
            outputs_u21  = logit_w2[u_idx]
            logit_o2 = net2(inputs1)
            logit_w12 = net2(inputs2)
            logit_w22 = net2(inputs3)
            outputs_u12 = logit_w12[u_idx]
            outputs_u22  = logit_w22[u_idx]
            pu = (torch.softmax(outputs_u11, dim=1) + torch.softmax(outputs_u21, dim=1) + torch.softmax(outputs_u12, dim=1) + torch.softmax(outputs_u22, dim=1)) / 4
            ptu = pu**(1/0.5) # temparature sharpening
            target_u = ptu / ptu.sum(dim=1, keepdim=True) # normalize
            target_u = target_u.detach().float() 
            
            px = torch.softmax(logit_o2[s_idx], dim=1) #+ torch.softmax(logit_w22[s_idx], dim=1)) / 2
            w_x = conf[indexes][s_idx]
            w_x = w_x.view(-1,1).float().cuda() 
            px = (1-w_x)*target_x + w_x*px              
            ptx = px**(1/0.5) # temparature sharpening           
            target_x = ptx / ptx.sum(dim=1, keepdim=True) # normalize           
            target_x = target_x.detach().float()      
            
            if logit_o2[u_idx].shape[0] > 0: 
                max_probs, targets_u1 = torch.max(torch.softmax(logit_o[u_idx], dim=1), dim=-1)    
                mask_u = max_probs.ge(0.99).float()
                u_idx2 = (mask_u == 1)
                unsupervised += torch.sum(mask_u).item()
                correct_u += torch.sum((targets_u1[u_idx2] == targets[u_idx][u_idx2])).item()
                update = indexes[u_idx][u_idx2]
                devide[update] = True
                p_label[update] = targets_u1[u_idx2].float()
        
        
        l = np.random.beta(4.0, 4.0)        
        l = max(l, 1-l)
        
        all_inputs = torch.cat([inputs2[s_idx], inputs3[s_idx], inputs2[u_idx], inputs3[u_idx]],dim=0)
        all_targets = torch.cat([target_x, target_x, target_u, target_u], dim=0)
        idx = torch.randperm(all_inputs.size(0))

        input_a, input_b = all_inputs, all_inputs[idx]
        target_a, target_b = all_targets, all_targets[idx]
        
        mixed_input = l * input_a + (1 - l) * input_b        
        mixed_target = l * target_a + (1 - l) * target_b
                
        logits = net(mixed_input)
        batch_size = target_x.shape[0]
        
        Lx, Lu = criterion_rb(logits[:batch_size*2], mixed_target[:batch_size*2], logits[batch_size*2:], mixed_target[batch_size*2:], epoch+batch_idx/num_iter)
        total_loss = Lx + Lu + LSloss(logit_s, labels_x.cuda())
        
        total_loss.backward()
        train_loss.update(total_loss.item(), inputs2.size(0))
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Epoch: [{epoch}][{elps_iters}/{tot_iters}] '
                  'Train loss: {train_loss.val:.4f} ({train_loss.avg:.4f}) '.format(
                      epoch=epoch, elps_iters=batch_idx,tot_iters=len(trainloader), 
                      train_loss=train_loss))
    conf_self = (conf_self - conf_self.min()) / (conf_self.max() - conf_self.min())
    
    return train_loss.avg, devide, p_label, conf_self


In [ ]:
args = config()
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainset, trainloader, testset, evalloader, class_num = preprocess(args)
net = models.ClusteringModel(models.__dict__['Resnet_CIFAR'](), class_num)
net2 = copy.deepcopy(net)
net_uc = copy.deepcopy(net)
net_embd = models.ContrastiveModel(models.__dict__['Resnet_CIFAR']())

try:
    state_dict = torch.load(args.o_model)
    state_dict2 = torch.load(args.e_model)
    net_uc.load_state_dict(state_dict)
    net_embd.load_state_dict(state_dict2, strict = True)
    net.load_state_dict(state_dict, strict = False)
    net2.load_state_dict(state_dict, strict = False)
    net.cluster_head = nn.ModuleList([nn.Linear(512, class_num) for _ in range(1)])
    net2.cluster_head = nn.ModuleList([nn.Linear(512, class_num) for _ in range(1)])
except:
    print("Check Model Directory!")
    exit(0)

if args.device == 'cuda':
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    net2 = torch.nn.DataParallel(net2, device_ids=range(torch.cuda.device_count()))
    net_uc = torch.nn.DataParallel(net_uc, device_ids=range(torch.cuda.device_count()))
    net_embd = torch.nn.DataParallel(net_embd, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True



In [ ]:

net.to(args.device)
net2.to(args.device)
net_uc.to(args.device)
net_embd.to(args.device)

optimizer1 = torch.optim.SGD(net.parameters(), args.lr, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=True)
optimizer2 = torch.optim.SGD(net2.parameters(), args.lr, momentum=args.momentum, weight_decay=args.weight_decay, nesterov=True)
criterion = criterion_rb()



In [ ]:
# Extract Pseudo Label
acc_uc, p_label= test(net_uc, evalloader, args.device, class_num)
print(acc_uc)
devide1 = extract_confidence(net_uc, p_label, evalloader, args.s_thr)
devide2 = extract_metric(net_embd, p_label, evalloader, args.n_num)
devide = extract_hybrid(devide1, devide2, p_label, evalloader)

In [ ]:
state = torch.load('./checkpoint/cifar10_ruc.t7')

In [ ]:
net.load_state_dict(state['net1'])

In [ ]:
net2.load_state_dict(state['net2'])

In [ ]:
acc, p_list = test_ruc(net, net2, evalloader, args.device, class_num)
print("accuracy: {}\n".format(acc))

In [ ]:
print(p_list)

## TRAIN GAN ON IT

In [ ]:
import gan_attack

In [ ]:
use_cuda=True
image_nc=3
epochs = 60
batch_size = 256
BOX_MIN = 0
BOX_MAX = 1
model_num_labels = 10
device = 'cuda'

In [ ]:
GAN = gan_attack.GAN_Attack(device, net, net2, model_num_labels, image_nc, BOX_MIN, BOX_MAX, 'cifar10')

In [ ]:
import sys

# Holding the original output object. i.e. console out
orig_stdout = sys.stdout

# Opening the file to write file deletion logs.
f = open('outgan_train-every256.txt', 'a+')

# Changing standard out to file out. 
sys.stdout = f
# This will write to the file. 
print("xyz") 

In [ ]:
# %%time
advGAN.train(evalloader, 600)

In [ ]:
# Closing the file.
f.close()


# replacing the original output format to stdout.
sys.stdout = orig_stdout

# This will print onto the console.
print("xyz") 

In [ ]:
ls


## Testing attack

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_2ncifar10_epoch_600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10')
print("accuracy: {}\n".format(acc))


In [ ]:
acc, p_list = test_ruc_adv_save(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10')

In [ ]:
acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.1, 'CIFAR10')
print("accuracy: {}\n".format(acc))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
from scipy.misc import toimage

MEAN = torch.tensor([0.4914, 0.4822, 0.4465]).cuda()
STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda()

# x = normalized_img * STD[:, None, None] + MEAN[:, None, None]

def pp(images, labels):
    MEAN = torch.tensor([0.485, 0.456, 0.406]).cuda()
    STD = torch.tensor([0.2023, 0.1994, 0.2010]).cuda()
    print(type(images[0]), type(labels))
    print(images.shape)
#     w = 
#     h = 10
    fig = plt.figure(figsize=(15, 15))
    columns = 11
    rows = 12
    for i in range(35):
        fig.add_subplot(rows, columns, i+1)
        img = images[i]
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5   # unnormalize
        img = img * STD[:, None, None] + MEAN[:, None, None]
        img = img.detach().cpu().numpy()
        img = np.clip(img, 0, 1)
#         img = (img * 255).astype(np.uint8)
#         img = img / 2 + 0.5
#         img = img / 2 + 0.5 
#         npimg = img.detach().cpu().numpy()   # convert from tensor
        
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
        plt.imshow(img.transpose(1, 2, 0))
#         plt.imshow(npimg)
#         plt.title('#{}: {}'.format(i, labels[i]))
    plt.show()

In [ ]:
def printall(evalloader):
    for batch_idx, (inputs, _, _, targets, indexes) in enumerate(evalloader):
        batchSize = inputs.size(0)
        targets, inputs = targets.to(device), inputs.to(device)
        perturbation = pretrained_G(inputs)
        perturbation = torch.clamp(perturbation, -0.25, 0.25)
        adv_imgs = perturbation + inputs
    #     print(inputs[0][0])
    #     print(adv_imgs[0][0])
        pp(inputs, targets)
        pp(adv_imgs, targets)
        pp(perturbation, targets)
        break

In [ ]:
nmi_l = []
ari_l = []
for j in range(10):
    acc = test_ruc_cls(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10', j)
    nmi_l.append(acc[0][-2])
    ari_l.append(acc[0][-1])

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
classes = [str(i) for i in range(10)]
plt.plot(classes, nmi_l)
print(nmi_l)
# plt.show()

In [ ]:
plt.plot(classes, ari_l)
print(ari_l)

In [ ]:
acc = test_ruc_cls(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10', 3)
print(acc)

# transferability

In [ ]:
#Base CC

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_cc_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.5, 'CIFAR10')
print("accuracy: {}\n".format(acc))


In [ ]:
#Mice

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_MICE_cifar-10_epoch_120.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.5, 'CIFAR10')
print("accuracy: {}\n".format(acc))


In [ ]:
#NNM

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_NNM_cifar-10_epoch_570.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.2, 'CIFAR10')
print("accuracy: {}\n".format(acc))


In [ ]:
#trasnfer scan

#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_SCAN_cifar-10_epoch_1200.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.2, 'CIFAR10')
print("accuracy: {}\n".format(acc))



In [ ]:
#trasnfer spice

#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/transferability/netG_SPICE_cifar-10_epoch600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

acc, p_list = test_ruc_adv(net, net2, evalloader, args.device, class_num, pretrained_G, 0.2, 'CIFAR10')
print("accuracy: {}\n".format(acc))



# additional Experiments

In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu
use_cuda=True
image_nc=3
batch_size = 128

gen_input_nc = image_nc
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = './models/netG_cc_2ncifar10_epoch_600.pth'
pretrained_G = models_clu.Generator(gen_input_nc, image_nc).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
acc, p_list, norm = test_ruc_adv_norm(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10')
acc2, nmi, ari = acc[-3], acc[-2], acc[-1]


In [ ]:
print(acc2, nmi, ari, norm)

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0, 0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
#     acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    acc, p_list, norm = test_ruc_adv_norm(net, net2, evalloader, args.device, class_num, pretrained_G, j, 'CIFAR10')
    acc2, nmi, ari = acc[-3], acc[-2], acc[-1]
#         print(clustering_stats)
    
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)

In [ ]:
# plt.plot(x, y, label = "line 1", linestyle="-")
# plt.plot(y, x, label = "line 2", linestyle="--")
# plt.plot(x, np.sin(x), label = "curve 1", linestyle="-.")
# plt.plot(x, np.cos(x), label = "curve 2", linestyle=":")

plt.plot(norm_l, nmi_l, label = "nmi", linestyle="-")
plt.plot(norm_l, ari_l, label = "ari", linestyle="-")
plt.plot(norm_l, acc_l, label = "acc", linestyle="-")
plt.xlabel("Perturbation Norm")
plt.ylabel("Performace")
plt.legend()
plt.grid()
plt.savefig('ruc_cifar10.png')
plt.show()

In [ ]:
print(norm_l)
print()
print(nmi_l)
print()
print(ari_l)
print()
print(acc_l)

In [ ]:
## same acc formuala

In [ ]:
acc, p_list, norm = test_ruc_adv_norm(net, net2, evalloader, args.device, class_num, pretrained_G, 0.25, 'CIFAR10')
acc2, nmi, ari = acc[-3], acc[-2], acc[-1]

In [ ]:
acc2, nmi, ari

In [ ]:
norm_l = []
nmi_l = []
ari_l = []
acc_l = []
# clamp = [j for j in range(0, 1, 0.02)]
# clamp = [j for j in np.arange(0, 1.05, 0.05)]
# clamp = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
clamp = [0, 0.25, 1.0]
print(clamp)

for j in clamp:
    torch.cuda.empty_cache()
#     acc2, mice_pi_acc, nmi, ari, norm = get_MiCE_adv_performance_norm(model, model_ema, elbo, full_loader, n_data, n_class, clamping=j)
    acc, p_list, norm = test_ruc_adv_norm(net, net2, evalloader, args.device, class_num, pretrained_G, j, 'CIFAR10')
    acc2, nmi, ari = acc[-3], acc[-2], acc[-1]
#         print(clustering_stats)
    
    print(f'clamp {j} avg norm: {norm}')
    print('NMI = {:.4f} ARI = {:.4f} ACC = {:.4f}'.format(nmi, ari, acc2))
    norm_l.append(norm)
    nmi_l.append(nmi)
    ari_l.append(ari)
    acc_l.append(acc2)